# Data Preparation

## Select Data - Eliminar Columnas
De la seccion de entendimiento de los datos, vimos que hay columnas que realmente no aportan mucha información, importancia o significancia al alcance de este proyecto, por lo que en esta parte, se eliminaran para contar unicamente con los datos que se trabajaran, estas columnas son:

- municipio_hechos : Al tener el 99% de sus datos como NaN y no aportar informacion
- competencia : Al tener un unico valor
- coordenadas (latitud y longitud) : El proyecto se enfocara en las alcaldias de la Ciudad de Mexico y el valor de coordenadas no estan dentro de
este scope
- colonia_datos : No esta dentro del scope de este proyecto
- fgj_colonia_registro : No esta dentro del scope de este proyecto

In [2]:
import pandas as pd

data_set = pd.read_csv("data/sourceData.csv")

In [3]:
columns_to_remove = ['municipio_hechos', 'competencia', 'latitud', 'longitud', 'colonia_datos', 'fgj_colonia_registro']
data_set = data_set.drop(columns_to_remove, axis=1)

data_set.head()

,idCarpeta,Anio_inicio,Mes_inicio,FechaInicio,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,Anio_hecho,Mes_hecho,FechaHecho,HoraHecho,HoraInicio,alcaldia_hechos
0,8324429.0,2019,Enero,04/01/2019,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62.0,FISICA,OFENDIDO,2018.0,Agosto,29/08/2018,12:00:00,12:19:00,ALVARO OBREGON
1,8324430.0,2019,Enero,04/01/2019,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38.0,FISICA,VICTIMA Y DENUNCIANTE,2018.0,Diciembre,15/12/2018,15:00:00,12:20:00,AZCAPOTZALCO
2,8324431.0,2019,Enero,04/01/2019,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42.0,FISICA,VICTIMA Y DENUNCIANTE,2018.0,Diciembre,22/12/2018,15:30:00,12:23:00,COYOACAN
3,8324435.0,2019,Enero,04/01/2019,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35.0,FISICA,VICTIMA Y DENUNCIANTE,2019.0,Enero,04/01/2019,06:00:00,12:27:00,IZTACALCO
4,8324438.0,2019,Enero,04/01/2019,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,NaN,FISICA,VICTIMA,2019.0,Enero,03/01/2019,20:00:00,12:35:00,IZTAPALAPA


## Outliers
Las columnas que cuentan con valores outliers son:
- edad: Valores mayores a 120 (258, 361 y 369) y 1899 valores de 0
- año_hechos: Un valor de 222 y valores del siglo pasado

In [4]:
valores_edad = data_set.groupby('Edad')['idCarpeta'].count()
valores_ahecho = data_set.groupby('Anio_hecho')['idCarpeta'].count()

### Outliers - Edad
El approach a seguir es el obtener un promedio de edad entre un limite inferior (1) y un limite superior (120)
Despues de obtener este promedio, reemplazamos los outliers (valores de 0, 258, 361 y 369) con este promedio, ya que el numero de valores en 0 es significativo (1899)

In [5]:
limite_inferior = 1
limite_superior = 120
edad_en_rango = data_set.loc[data_set['Edad'].between(limite_inferior, limite_superior), 'Edad']
promedio_edad = edad_en_rango.mean()
print(promedio_edad)

promedio_edad = round(promedio_edad)
print(promedio_edad)

38.944485754445886
39


In [6]:
data_set['Edad'] = data_set['Edad'].mask(~data_set['Edad'].between(limite_inferior, limite_superior), promedio_edad)
print(data_set.groupby('Edad')['idCarpeta'].count())

Edad
1.0       942
2.0      1206
3.0      1685
4.0      1837
5.0      1848
         ... 
107.0       1
110.0       1
111.0       1
114.0       1
120.0       2
Name: idCarpeta, Length: 110, dtype: int64


## Outliers - Año Hechos
Se borran las filas con valores menores a 2000

In [7]:
data_set = data_set[data_set['Anio_hecho'] >= 2000]
print(data_set.groupby('Anio_hecho')['idCarpeta'].count())

Anio_hecho
2000.0       118
2001.0        93
2002.0       212
2003.0        68
2004.0        84
2005.0        84
2006.0       108
2007.0       129
2008.0       231
2009.0       254
2010.0       353
2011.0       430
2012.0       531
2013.0       541
2014.0       880
2015.0      1085
2016.0      1732
2017.0      3457
2018.0     17888
2019.0    261051
2020.0    214736
2021.0    235849
2022.0    233278
Name: idCarpeta, dtype: int64


## Cleaning Data
Se borran las filas con valores a CDMX y FUERA DE CDMX para las alcaldias
Ya que este proyecto se enfoca unicamente en alcaldias dentro de la ciudad

In [8]:
data_set = data_set[~data_set['alcaldia_hechos'].isin(['CDMX', 'FUERA DE CDMX'])]
print(data_set['alcaldia_hechos'].unique())

['ALVARO OBREGON' 'AZCAPOTZALCO' 'COYOACAN' 'IZTACALCO' 'IZTAPALAPA'
 'GUSTAVO A. MADERO' 'VENUSTIANO CARRANZA' 'XOCHIMILCO' 'MIGUEL HIDALGO'
 'CUAUHTEMOC' 'BENITO JUAREZ' 'TLAHUAC' 'TLALPAN' 'LA MAGDALENA CONTRERAS'
 'CUAJIMALPA DE MORELOS' 'MILPA ALTA']


## Missing Data
Las columnas con valores NaN son las siguientes:
- Sexo : 180,609 registros
- TipoPersona : 6512 registros
- CalidadJuridica: 1 registro
- Anio_hecho, Mes_hecho, FechaHecho : 370 registros
- HoraHecho : 361 registros
- HoraInicio : 1
- colonia_datos : 69,602 registros

In [9]:
for column in data_set.columns:
    if(data_set[column].isnull().sum() > 0):
        print(column + ' : ' + str(data_set[column].isnull().sum()))

Sexo : 179449
TipoPersona : 6426
CalidadJuridica : 1
HoraHecho : 1
HoraInicio : 1


### Grupo 1 - Columnas con menos de 1000 registros Nan
El approach para este grupo es borrar los registros, ya que son de un numero muy bajo
Las columnas a borrar sus registros con valores nulos son:
- CalidadJuridica
- Anio_hecho
- Mes_hecho
- FechaHecho
- HoraHecho
- HoraInicio

In [10]:
data_set = data_set.dropna(subset=['CalidadJuridica', 'Anio_hecho', 'Mes_hecho', 'FechaHecho', 'HoraHecho', 'HoraInicio'])
for column in data_set.columns:
    if(data_set[column].isnull().sum() > 0):
        print(column + ' : ' + str(data_set[column].isnull().sum()))

Sexo : 179449
TipoPersona : 6426


### Grupo 2 - Columnas con mas de 1000 registros Nan
Para estas columnas, el approach es sustituir los valores Nulos con la palabra 'Desconocido'
Las columnas a ser modificadas son:
- Sexo
- TipoPersona
- colonia_datos

In [11]:
data_set[['Sexo', 'TipoPersona']] = data_set[['Sexo', 'TipoPersona']].fillna('Desconocido')
for column in data_set.columns:
    if(data_set[column].isnull().sum() > 0):
        print(column + ' : ' + str(data_set[column].isnull().sum()))

## Data Transformation
En esta seccion, se modificaran las columnas de formato HH:MM:SS a HH:MM
Estas columnas son:
- HoraHecho
- HoraInicio

In [12]:
data_set[['HoraHecho', 'HoraInicio']] = data_set[['HoraHecho', 'HoraInicio']].apply(pd.to_datetime)
data_set['HoraHecho'] = data_set['HoraHecho'].dt.strftime('%H:%M')
data_set['HoraInicio'] = data_set['HoraInicio'].dt.strftime('%H:%M')
data_set.head()

,idCarpeta,Anio_inicio,Mes_inicio,FechaInicio,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,Anio_hecho,Mes_hecho,FechaHecho,HoraHecho,HoraInicio,alcaldia_hechos
0,8324429.0,2019,Enero,04/01/2019,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62.0,FISICA,OFENDIDO,2018.0,Agosto,29/08/2018,12:00,12:19,ALVARO OBREGON
1,8324430.0,2019,Enero,04/01/2019,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38.0,FISICA,VICTIMA Y DENUNCIANTE,2018.0,Diciembre,15/12/2018,15:00,12:20,AZCAPOTZALCO
2,8324431.0,2019,Enero,04/01/2019,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42.0,FISICA,VICTIMA Y DENUNCIANTE,2018.0,Diciembre,22/12/2018,15:30,12:23,COYOACAN
3,8324435.0,2019,Enero,04/01/2019,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35.0,FISICA,VICTIMA Y DENUNCIANTE,2019.0,Enero,04/01/2019,06:00,12:27,IZTACALCO
4,8324438.0,2019,Enero,04/01/2019,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,39.0,FISICA,VICTIMA,2019.0,Enero,03/01/2019,20:00,12:35,IZTAPALAPA


### Crear Columna Dias Transcurridos
Con base a las columnas FechaInicio y FechaHecho, obtenemos los dias transcurridos entre la fecha en que se reporto el delito y la fecha en que ocurrio, para darnos una idea de si estos dias tienen un patron respecto a sexo o edad (hipotesis).

Una vez realizado esto, las columnas de Anio_inicio, Mes_inicio y FechaInicio no seran de utilidad ya que para realizar las predicciones las fechas importantes son las de los hechos

In [13]:
fechas_inicio = pd.to_datetime(data_set['FechaInicio'], format="%d/%m/%Y")
fechas_hechos = pd.to_datetime(data_set['FechaHecho'], format="%d/%m/%Y")

data_set.insert(13, 'Dias_transcurridos', (fechas_inicio - fechas_hechos).dt.days)

# Borrar fechas inicio
columns_to_remove = ['Anio_inicio', 'Mes_inicio', 'FechaInicio']
data_set = data_set.drop(columns_to_remove, axis=1)

data_set.head()

,idCarpeta,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,Anio_hecho,Mes_hecho,FechaHecho,Dias_transcurridos,HoraHecho,HoraInicio,alcaldia_hechos
0,8324429.0,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62.0,FISICA,OFENDIDO,2018.0,Agosto,29/08/2018,128,12:00,12:19,ALVARO OBREGON
1,8324430.0,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38.0,FISICA,VICTIMA Y DENUNCIANTE,2018.0,Diciembre,15/12/2018,20,15:00,12:20,AZCAPOTZALCO
2,8324431.0,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42.0,FISICA,VICTIMA Y DENUNCIANTE,2018.0,Diciembre,22/12/2018,13,15:30,12:23,COYOACAN
3,8324435.0,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35.0,FISICA,VICTIMA Y DENUNCIANTE,2019.0,Enero,04/01/2019,0,06:00,12:27,IZTACALCO
4,8324438.0,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,39.0,FISICA,VICTIMA,2019.0,Enero,03/01/2019,1,20:00,12:35,IZTAPALAPA


### Crear Columna Semanas
Con base a la columna FechaHecho, tomaremos el valor mas viejo (el minimo de la fecha) y comenzaremos un counter de semana en uno, por cada semana transcurrida aumentaremos el counter a un valor de 1 para crear un timelapse semanal desde el hecho mas viejo al mas reciente para posteriormente realizar predicciones de delitos por semana.
1. Obtener el valor minimo de FechaHecho
2. Comenzar el counter de semanas
1. Por cada semana transcurrida aumentar el counter
2. Asignar el valor de counter correspondiente a la FechaHecho en una columna 'abs_semana'
3. Eliminar la columna Anio_hecho ya que no aporta mucho

Se descubrio que la fecha mas vieja del data_set es el primero de enero del 2000, si se quiere realizar algun modelo predictivo desde una fecha mas reciente a esta, por ejemplo, desde el 2015, se debera crear un offset para evitar errores, un ejemplo seria:
- La semana uno original es 01/01/2000
- El modelo predictivo que se desea crear (ejemplo) es a partir del 01/01/2015
- Han pasada 782 semanas entre ambas fechas por lo que la fecha 01/01/2015 tiene un valor de 782 en la columna count_semanas
- Se crea un offset de 782 es decir (count_semanas) - 781 para tomar a la fecha 01/01/2015 como la mas vieja
- NOTA: Ajustar el numero de 782 por la fecha en que se desea comenzar el modelo

In [14]:
oldest_date = fechas_hechos.min()
data_set.insert(11, 'count_semanas', (fechas_hechos - oldest_date).dt.days // 7 + 1)
data_set = data_set.drop(['Anio_hecho'], axis=1)
data_set.head()

,idCarpeta,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,Mes_hecho,FechaHecho,Dias_transcurridos,count_semanas,HoraHecho,HoraInicio,alcaldia_hechos
0,8324429.0,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62.0,FISICA,OFENDIDO,Agosto,29/08/2018,128,974,12:00,12:19,ALVARO OBREGON
1,8324430.0,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38.0,FISICA,VICTIMA Y DENUNCIANTE,Diciembre,15/12/2018,20,990,15:00,12:20,AZCAPOTZALCO
2,8324431.0,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42.0,FISICA,VICTIMA Y DENUNCIANTE,Diciembre,22/12/2018,13,991,15:30,12:23,COYOACAN
3,8324435.0,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35.0,FISICA,VICTIMA Y DENUNCIANTE,Enero,04/01/2019,0,992,06:00,12:27,IZTACALCO
4,8324438.0,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,39.0,FISICA,VICTIMA,Enero,03/01/2019,1,992,20:00,12:35,IZTAPALAPA


### Crear Columna Rango Edad
Con base al valor de la columna Edad, se creara una variable categorica descriptiva generando 6 rangos diferentes de edad (Niño, Adolescente, y más), con la finalidad de tener un entendimiento mejor de los datos y poder interpretar mejor los resultados del modelo a crear posteriormente

In [15]:
from mapeo_poblacion import obtener_rango_edad

data_set.insert(loc=5, column='rango_edad', value=data_set['Edad'].apply(obtener_rango_edad))
data_set.head()

,idCarpeta,Delito,Categoria,Sexo,Edad,rango_edad,TipoPersona,CalidadJuridica,Mes_hecho,FechaHecho,Dias_transcurridos,count_semanas,HoraHecho,HoraInicio,alcaldia_hechos
0,8324429.0,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62.0,Adultos mayores,FISICA,OFENDIDO,Agosto,29/08/2018,128,974,12:00,12:19,ALVARO OBREGON
1,8324430.0,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38.0,Adultos jóvenes,FISICA,VICTIMA Y DENUNCIANTE,Diciembre,15/12/2018,20,990,15:00,12:20,AZCAPOTZALCO
2,8324431.0,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42.0,Adultos de mediana edad,FISICA,VICTIMA Y DENUNCIANTE,Diciembre,22/12/2018,13,991,15:30,12:23,COYOACAN
3,8324435.0,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35.0,Adultos jóvenes,FISICA,VICTIMA Y DENUNCIANTE,Enero,04/01/2019,0,992,06:00,12:27,IZTACALCO
4,8324438.0,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,39.0,Adultos jóvenes,FISICA,VICTIMA,Enero,03/01/2019,1,992,20:00,12:35,IZTAPALAPA


### Columnas Sexo y tipo de persona Numerico

In [16]:
from mapeo_poblacion import mapeo_sexo, mapeo_tipo_persona

data_set.insert(loc=5, column='Sexo_num', value=data_set['Sexo'].apply(mapeo_sexo))
data_set.insert(loc=8, column='TipoPersona_num', value=data_set['TipoPersona'].apply(mapeo_tipo_persona))

## Data Integration

### Poblacion Alcaldias
En esta fase nos centraremos en agregar información externa proveniente de otras bases de datos y fuentes de información
Para esto, integraremos datos obtenidos de las estadisticas de Poblacion del INEGI para mostrar la poblacion por alcaldia, agregando una nueva columna de poblacion dependiendo el valor de la columna de alcaldia_hechos
Para realizar este mapeo entre la alcaldia de los hechos y su poblacion, se llevo a cabo un script en Python

In [17]:
from mapeo_poblacion import get_population

data_set.insert(loc=14, column='poblacion_alcaldia', value=data_set['alcaldia_hechos'].apply(get_population))
data_set.head()

,idCarpeta,Delito,Categoria,Sexo,Edad,Sexo_num,rango_edad,TipoPersona,TipoPersona_num,CalidadJuridica,Mes_hecho,FechaHecho,Dias_transcurridos,count_semanas,poblacion_alcaldia,HoraHecho,HoraInicio,alcaldia_hechos
0,8324429.0,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62.0,5,Adultos mayores,FISICA,5,OFENDIDO,Agosto,29/08/2018,128,974,759137,12:00,12:19,ALVARO OBREGON
1,8324430.0,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38.0,10,Adultos jóvenes,FISICA,5,VICTIMA Y DENUNCIANTE,Diciembre,15/12/2018,20,990,432205,15:00,12:20,AZCAPOTZALCO
2,8324431.0,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42.0,5,Adultos de mediana edad,FISICA,5,VICTIMA Y DENUNCIANTE,Diciembre,22/12/2018,13,991,614447,15:30,12:23,COYOACAN
3,8324435.0,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35.0,5,Adultos jóvenes,FISICA,5,VICTIMA Y DENUNCIANTE,Enero,04/01/2019,0,992,404695,06:00,12:27,IZTACALCO
4,8324438.0,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,39.0,5,Adultos jóvenes,FISICA,5,VICTIMA,Enero,03/01/2019,1,992,1835486,20:00,12:35,IZTAPALAPA


### Categorización de Delitos
Actualmente el dataset cuenta con 16 tipos diferentes de Categorias y 310 tipos diferentes de Delitos, al ser tantos valores unicos se dificulta el
analisis conciso de este tipo de delitos y los 16 tipos de categoria son algo ambigüos como "Delito de bajo Impacto".

Para esto se ha utilizado de un algoritmo open source que emplea técnicas avanzadas de procesamiento de lenguaje natural, para clasificar distintos
tipos de delitos en funcion de su nivel de riesgo y severidad. Este algoritmo analiza y evalua la gravedad de cada delito, permitiendo establecer
rangos que van desde situaciones de menor riesgo (valor de 1) hasta aquellas que presentan un mayor nivel de peligro (valor de 10). El objetivo de esta
herramienta es proporcionar una clasificacion precisa y confiable de los delitos.

De esta forma la columna 'Categoria' deja de ser util y puede ser eliminada.

In [18]:
numero_categorias = len(pd.unique(data_set['Categoria']))
numero_delitos = len(pd.unique(data_set['Delito']))
print(f"Categorias Distintas =  {numero_categorias}; Delitos Distintos =  {numero_delitos}")

Categorias Distintas =  16; Delitos Distintos =  308


In [19]:
from mapeo_crimen import get_crime_score

data_set.insert(loc=2, column='riesgo_delito', value=data_set['Delito'].apply(get_crime_score))
data_set = data_set.drop('Categoria', axis=1)
data_set.head()

,idCarpeta,Delito,riesgo_delito,Sexo,Edad,Sexo_num,rango_edad,TipoPersona,TipoPersona_num,CalidadJuridica,Mes_hecho,FechaHecho,Dias_transcurridos,count_semanas,poblacion_alcaldia,HoraHecho,HoraInicio,alcaldia_hechos
0,8324429.0,FRAUDE,5,Masculino,62.0,5,Adultos mayores,FISICA,5,OFENDIDO,Agosto,29/08/2018,128,974,759137,12:00,12:19,ALVARO OBREGON
1,8324430.0,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",8,Femenino,38.0,10,Adultos jóvenes,FISICA,5,VICTIMA Y DENUNCIANTE,Diciembre,15/12/2018,20,990,432205,15:00,12:20,AZCAPOTZALCO
2,8324431.0,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,10,Masculino,42.0,5,Adultos de mediana edad,FISICA,5,VICTIMA Y DENUNCIANTE,Diciembre,22/12/2018,13,991,614447,15:30,12:23,COYOACAN
3,8324435.0,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,3,Masculino,35.0,5,Adultos jóvenes,FISICA,5,VICTIMA Y DENUNCIANTE,Enero,04/01/2019,0,992,404695,06:00,12:27,IZTACALCO
4,8324438.0,ROBO DE MOTOCICLETA SIN VIOLENCIA,3,Masculino,39.0,5,Adultos jóvenes,FISICA,5,VICTIMA,Enero,03/01/2019,1,992,1835486,20:00,12:35,IZTAPALAPA


In [20]:
data_set.to_csv('data/data_preparation.csv', index=False)